In [ ]:
import math
import numpy as np
import pandas as pd
 
def group(df,attr,cls_attr,rows):
    count={}
    rownums={}
    for r in rows:
        att=df.loc[r][attr]
        print(att)
        cls=df.loc[r][cls_attr]
        if att not in count:
            count[att]={}
            count[att]['TOTAL']=0
            rownums[att]=[]
#        print(count)
#        print(rownums)
        if cls not in count[att]:
            count[att][cls]=0
        count[att]['TOTAL']+=1
        count[att][cls]+=1
        rownums[att].append(r)
#        print(count)
#        print(rownums)
    return (count,rownums)
    
def compute_entropy(df,attr,cls_attr,rows):
    entropy=0
    count,rownums=group(df,attr,cls_attr,rows)
    for val in count:
        temp=0
        tot=count[val]['TOTAL']
        for cls in count[val]:
            if cls != 'TOTAL':
                v=count[val][cls]
                temp+=-v/tot*math.log(v/tot)
        entropy+=tot/len(rows)*temp
    return(entropy,rownums)
    
def create_tree(df,avail_attr,cls_attr,rows):
    if len(df.loc[rows,cls_attr].unique()) == 1:
        return df.loc[rows[0],cls_attr]
    elif len(avail_attr) == 0:
        return DEFAULT
    min_entropy=np.inf
    min_rownums=0
    min_attr=''
    for col in avail_attr:
        entropy,rownums=compute_entropy(df,col,cls_attr,rows)
        if entropy < min_entropy:
            min_entropy=entropy
            min_rownums=rownums
            min_attr=col
    avail_attr.remove(min_attr)
    node=(min_attr,min_rownums)
    for val in node[1]:
        node[1][val]=create_tree(df,avail_attr,cls_attr,node[1][val])
    return node    

def print_tree(tree,rule):
    if isinstance(tree,str):
        print(rule+' then '+tree)
    else:
        for val in tree[1]:
            print_tree(tree[1][val],rule+', '+tree[0]+' = '+val)
            
def predict(tree,test_data):
    for r in range(len(test_data)):
        node=tree
        prediction=' '
        while not isinstance(node,str):
            att=node[0]
            val=test_data.loc[r,att]
            if val in node[1]:
                node=node[1][val]
            else:
                prediction=DEFAULT
                break
        if isinstance(node,str):
            prediction=node
        print(list(test_data.loc[r]))
        print(prediction)
                
    
df = pd.read_csv('weatherFull.csv')   
DEFAULT=df.iloc[0,-1]

tree=create_tree(df,list(df.columns[:-1]),df.columns[-1],range(len(df)))

print_tree(tree,'If ')

test = pd.read_csv('test.csv')
predict(tree,test)